In [1]:
from tensorflow.keras.datasets import boston_housing
from tensorflow import keras
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

2023-03-20 10:41:59.437555: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load data

(train_data, train_targets), (test_data, test_trgets) = boston_housing.load_data()

In [3]:
train_data.shape

(404, 13)

In [4]:
test_data.shape

(102, 13)

In [5]:
scaler = StandardScaler()

In [6]:
scaler.fit(train_data)

StandardScaler()

In [7]:
scaler.mean_

array([3.74511057e+00, 1.14801980e+01, 1.11044307e+01, 6.18811881e-02,
       5.57355941e-01, 6.26708168e+00, 6.90106436e+01, 3.74027079e+00,
       9.44059406e+00, 4.05898515e+02, 1.84759901e+01, 3.54783168e+02,
       1.27408168e+01])

In [8]:
scaled_train_data = scaler.transform(train_data)
scaled_test_data = scaler.transform(test_data)

In [9]:
def build_model():
    model = keras.Sequential(
        [
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(64, activation="relu"),
            keras.layers.Dense(1)
        ]
    )
    
    model.compile(optimizer="rmsprop", loss="mse", metrics=["mae"])
    
    return model

In [13]:
train_targets.shape

(404,)

In [14]:
k = 4
num_val_samples = len(scaled_train_data)//k
num_epochs = 100
all_scores = []

for i in range(k):
    print(f"Processing fold #{i}")
    
    val_data = scaled_train_data[i*num_val_samples: (i+1)*num_val_samples]
    val_targets = train_targets[i*num_val_samples: (i+1)*num_val_samples]
    
    partial_train_data = np.concatenate(
        [
            scaled_train_data[:i*num_val_samples],
            scaled_train_data[(i+1)*num_val_samples:]
        ],
        axis=0
    )
    
    partial_train_targets = np.concatenate(
        [
            train_targets[:i*num_val_samples],
            train_targets[(i+1)*num_val_samples:]
        ],
        axis=0
    )
    
    model = build_model()
    model.fit(
        partial_train_data,
        partial_train_targets,
        epochs=num_epochs,
        batch_size=16,
        verbose=0
    )
    
    val_mse, val_mae = model.evaluate(val_data, val_targets)

Processing fold #0
4/4 [==============================] - 0s 863us/step - loss: 7.8967 - mae: 2.1486
Processing fold #1
4/4 [==============================] - 0s 895us/step - loss: 9.6416 - mae: 2.3914
Processing fold #2
4/4 [==============================] - 0s 841us/step - loss: 13.7635 - mae: 2.4529
Processing fold #3
4/4 [==============================] - 0s 881us/step - loss: 12.5569 - mae: 2.6496


In [15]:
val_mse

12.556857109069824